<a href="https://colab.research.google.com/github/Lucas-Jimenez-uribe/telecom_x/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [59]:
import pandas as pd
import numpy as np


In [60]:
df = pd.read_json('/content/TelecomX_Data.json')
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [61]:
df.shape

(7267, 6)

#🔧 Transformación

In [74]:
df

,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,internet.OnlineSecurity,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,female,0,yes,yes,9,yes,no,dsl,no,yes,no,yes,yes,no,one year,yes,mailed check,65.60,593.30
1,male,0,no,no,9,yes,yes,dsl,no,no,no,no,no,yes,month-to-month,no,mailed check,59.90,542.40
2,male,0,no,no,4,yes,no,fiber optic,no,no,yes,no,no,no,month-to-month,yes,electronic check,73.90,280.85
3,male,1,yes,no,13,yes,no,fiber optic,no,yes,yes,no,yes,yes,month-to-month,yes,electronic check,98.00,1237.85
4,female,1,yes,no,3,yes,no,fiber optic,no,no,no,yes,yes,no,month-to-month,yes,mailed check,83.90,267.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7262,female,0,no,no,13,yes,no,dsl,yes,no,no,yes,no,no,one year,no,mailed check,55.15,742.90
7263,male,0,yes,no,22,yes,yes,fiber optic,no,no,no,no,no,yes,month-to-month,yes,electronic check,85.10,1873.70
7264,male,0,no,no,2,yes,no,dsl,no,yes,no,no,no,no,month-to-month,yes,mailed check,50.30,92.75
7265,male,0,yes,yes,67,yes,no,dsl,yes,no,yes,yes,no,yes,two year,no,mailed check,67.85,4627.65


In [ ]:
df.info()

In [ ]:
col_inicio = df.columns.get_loc('customer')

# Seleccionar todas las columnas desde esa en adelante
cols_a_normalizar = df.columns[col_inicio:]

# Extraer esas columnas como diccionarios por fila
registros = df[cols_a_normalizar].to_dict(orient='records')

# Normalizar
df = df_normalizado = pd.json_normalize(registros)

df.head()

In [65]:
df.duplicated().sum()
df = df.drop_duplicates()

In [ ]:
df.isnull().sum()

In [ ]:
df['account.Charges.Total'].unique()

df['account.Charges.Total'] = df['account.Charges.Total'].replace('', np.nan)

df['account.Charges.Total'] = df['account.Charges.Total'].str.strip()

df['account.Charges.Total'] = df['account.Charges.Total'].astype(float)



In [ ]:
# Aplicar str.lower() y str.strip() a todas las columnas de texto
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].str.lower().str.strip()


In [ ]:
# Ver cuántas veces aparecen en cada columna
valores_nulos_falsos = ['n/a', 'na', 'null', 'none', '']

for valor in valores_nulos_falsos:
    print(f"--- Buscando '{valor}' ---")
    print((df == valor).sum())

In [ ]:
df.info()

#📊 Carga y análisis

#📄Informe final